# Processing
Demonstration of Processing endpoints.

In [1]:
import utils.DemoClient as client
import jwt
import json

## Initialise

### Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [2]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "demo.eoepca.org"
platform_domain = "test." + base_domain
base_url = "https://" + platform_domain
demo = client.DemoClient(base_url)
demo.register_client()
demo.save_state()

State loaded from file: {'client_id': '20f9737b-80a0-4087-8aa0-68bde5a23e39', 'client_secret': '41294b47-a177-45e0-8a2b-5497f8654ba1'}
client_id: 20f9737b-80a0-4087-8aa0-68bde5a23e39 [REUSED]
Client state saved to file: state.json


### User
User authenticates and the client receives an ID Token (JWT) that represents the user.

In [3]:
#-------------------------------------------------------------------------------
# Authenticate as UserA and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="bob"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

[Request] GET => https://test.demo.eoepca.org/.well-known/uma2-configuration
<Response [200]>
token_endpoint: https://test.demo.eoepca.org/oxauth/restv1/token
[Request] POST => https://test.demo.eoepca.org/oxauth/restv1/token


## ADES
The ADES provides user-specific endpoints, using a URL path prefix.

In [4]:
# ADES URLs
ades_base_url = "http://ades.demo.eoepca.org" #"http://ades." + platform_domain
ades_wps_url = ades_base_url + "/" + USER_NAME + "/zoo"; print("ADES WPS endpoint:", ades_wps_url)
ades_proc_url = ades_base_url + "/" + USER_NAME + "/wps3"; print("ADES API Processes endpoint:", ades_proc_url)

ADES WPS endpoint: http://ades.demo.eoepca.org/bob/zoo
ADES API Processes endpoint: http://ades.demo.eoepca.org/bob/wps3


### ADES API Processes

In [5]:
ades_access_token = None
app_name = "burned-area-1_0"

#### ADES: List Processes

In [6]:
# API Processes - List Processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.demo.eoepca.org/bob/wps3/processes
[Process List] = 200 (OK)
Processes: ['nhi-0_0_3', 'DeployProcess', 'GetStatus', 'burned-area-1_0', 'burned-area-0_5', 'snuggs-0_3_0', 'UndeployProcess', 's-expression-0_0_2']


#### ADES: Deploy Application

In [7]:
# API Processes - Deploy Application
response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/burned-area-1.0/app-deploy-body-cwl.json", id_token=user_id_token, access_token=ades_access_token)

Application details read from file: ../data/burned-area-1.0/app-deploy-body-cwl.json
[Request] POST => http://ades.demo.eoepca.org/bob/wps3/processes
[Deploy Response] = 201 (Created)


In [8]:
# Check deploy - list processes again
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

[Request] GET => http://ades.demo.eoepca.org/bob/wps3/processes
[Process List] = 200 (OK)
Processes: ['nhi-0_0_3', 'DeployProcess', 'GetStatus', 'burned-area-1_0', 'burned-area-0_5', 'snuggs-0_3_0', 'UndeployProcess', 's-expression-0_0_2']


#### ADES: Get Application Details

In [9]:
# API Processes - Get Application Details
response, ades_access_token = demo.proc_get_app_details(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

[Request] GET => http://ades.demo.eoepca.org/bob/wps3/processes/burned-area-1_0
[App Details] = 200 (OK)
{
  "process": {
    "id": "burned-area-1_0",
    "title": "Burned area",
    "abstract": "Burned area detection based on NDVI/NDWI thresholds",
    "version": "1.0.0.0",
    "jobControlOptions": [
      "sync-execute",
      "async-execute",
      "dismiss"
    ],
    "outputTransmission": [
      "value",
      "reference"
    ],
    "links": [
      {
        "rel": "canonical",
        "type": "application/json",
        "title": "Execute End Point",
        "href": "/watchjob/processes/burned-area-1_0/jobs/"
      }
    ],
    "inputs": [
      {
        "id": "pre_event",
        "title": "Pre event input product reference",
        "abstract": "Pre event input product reference",
        "minOccurs": "1",
        "maxOccurs": "1",
        "input": {
          "literalDataDomains": [
            {
              "dataType": {
                "name": "string"
              },
  

#### ADES: Execute Application

In [10]:
# API Processes - Execute Application
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/burned-area-1.0/app-execute-body.json", id_token=user_id_token, access_token=ades_access_token)

Application execute details read from file: ../data/burned-area-1.0/app-execute-body.json
[Request] POST => http://ades.demo.eoepca.org/bob/wps3/processes/burned-area-1_0/jobs
[Execute Response] = 201 (Created) => job=/bob/watchjob/processes/burned-area-1_0/jobs/2bc7d1e8-9a01-11ec-9e69-8a8147a03f5b


#### ADES: Job Status

In [11]:
# API Processes - Job Status
response, ades_access_token, status = demo.proc_get_job_status(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

[Request] GET => http://ades.demo.eoepca.org/bob/watchjob/processes/burned-area-1_0/jobs/2bc7d1e8-9a01-11ec-9e69-8a8147a03f5b
[Job Status] = 200 (OK) => 02/03/2022 08:17:17 - running
{
  "jobID": "2bc7d1e8-9a01-11ec-9e69-8a8147a03f5b",
  "started": "2022-03-02T08:17:16.180333Z",
  "status": "running",
  "message": "running",
  "progress": "0",
  "links": [
    {
      "href": "http://ades.demo.eoepca.org/bob/watchjob/processes/burned-area-1_0/jobs/2bc7d1e8-9a01-11ec-9e69-8a8147a03f5b",
      "rel": "self",
      "type": "application/json",
      "title": "get Status"
    }
  ]
}


In [ ]:
# API Processes - Job Status (keep polling for completion)
response, ades_access_token, status = demo.proc_poll_job_completion(ades_base_url, job_location_path, interval=10, id_token=user_id_token, access_token=ades_access_token)

[Job Status] = 200 (OK) => 02/03/2022 08:17:18 - running
[Job Status] = 200 (OK) => 02/03/2022 08:17:28 - running
[Job Status] = 200 (OK) => 02/03/2022 08:17:39 - running
[Job Status] = 200 (OK) => 02/03/2022 08:17:50 - running
[Job Status] = 200 (OK) => 02/03/2022 08:18:00 - running
[Job Status] = 200 (OK) => 02/03/2022 08:18:11 - running
[Job Status] = 200 (OK) => 02/03/2022 08:18:22 - running
[Job Status] = 200 (OK) => 02/03/2022 08:18:32 - running
[Job Status] = 200 (OK) => 02/03/2022 08:18:43 - running
[Job Status] = 200 (OK) => 02/03/2022 08:18:54 - running
[Job Status] = 200 (OK) => 02/03/2022 08:19:05 - running
[Job Status] = 200 (OK) => 02/03/2022 08:19:15 - running
[Job Status] = 200 (OK) => 02/03/2022 08:19:26 - running
[Job Status] = 200 (OK) => 02/03/2022 08:19:37 - running
[Job Status] = 200 (OK) => 02/03/2022 08:19:47 - running
[Job Status] = 200 (OK) => 02/03/2022 08:19:58 - running
[Job Status] = 200 (OK) => 02/03/2022 08:20:09 - running
[Job Status] = 200 (OK) => 02/0

In [ ]:
print(response.status_code, response.headers, response.text)

#### ADES: Job Result

In [ ]:
# API Processes - Job Result
response, ades_access_token, stacCatalogUri = demo.proc_get_job_result(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

#### ADES: List Jobs

In [ ]:
# API Processes - List Jobs
response, ades_access_token, job_ids = demo.proc_list_jobs(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print("Jobs:", job_ids)

#### ADES: Undeploy Application

In [ ]:
# API Processes - Undeploy Application
response, ades_access_token = demo.proc_undeploy_application(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check undeploy - list processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

## Workspace

In [ ]:
workspace_url = "https://workspace-api." + base_domain
workspace_access_token = None

### Workspace: Get Details

In [ ]:
# Workspace - Get Details
workspace_name = "demo-user-" + USER_NAME.lower()
response, workspace_access_token = demo.workspace_get_details(workspace_url, workspace_name, id_token=user_id_token, access_token=workspace_access_token)
workspace_details = response.json()
print(json.dumps(workspace_details, indent = 2))

### Inspect S3 Bucket

In [ ]:
# Bucket details
bucket_name = workspace_details["storage"]["credentials"]["bucketname"]
s3_access = workspace_details["storage"]["credentials"]["access"]
s3_secret = workspace_details["storage"]["credentials"]["secret"]

In [ ]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# List bucket contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(' ->', obj)